In [6]:
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph
import json

In [7]:
nodes_path = "../file_data/fb-pages-food.nodes"
edges_path= "../file_data/fb-pages-food.edges"

nodes_df = pd.read_csv(
    nodes_path, sep=",", header=None, names=["node", "name", "new_id"], skiprows=1
)
edges_df = pd.read_csv(edges_path, sep=",", header=None, names=["source", "target"])

nodes_df["new_id"] = nodes_df["new_id"].astype(str)
edges_df["source"] = edges_df["source"].astype(str)
edges_df["target"] = edges_df["target"].astype(str)

In [8]:
G = nx.Graph()

for index, row in nodes_df.iterrows():
    G.add_node(row["new_id"], name=row["name"], original_id=row["node"])


for index, row in edges_df.iterrows():
    G.add_edge(row["source"], row["target"])

In [9]:
max_cliques = list(nx.find_cliques(G))

In [10]:
graph_data = json_graph.node_link_data(G)

output_data= {
    "graph_data": graph_data,
    "max_cliques": max_cliques
}

In [11]:
with open("../graph_data/graph_data_cliques.json", "w") as f:
    json.dump(output_data, f, indent=4)